In [11]:
from llama_index.embeddings import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [19]:
embeddings = embed_model.get_text_embedding("Some text to embed.")
print(len(embeddings))
print(embeddings[:5])

768
[0.02955850027501583, 0.006734931375831366, 0.023973623290657997, 0.0575539767742157, 0.07338541001081467]


In [1]:
from llama_index.embeddings import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")
# embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embeddings = embed_model.get_text_embedding("Some text to embed.")
print(len(embeddings))
print(embeddings[:5])

In [3]:
# embed_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
embeddings = embed_model.get_text_embedding("Some text to embed.")
print(len(embeddings))
print(embeddings[:5])

384
[0.018472013995051384, -0.0028620639350265265, -0.0009641858050599694, -0.012189453467726707, 0.0160993505269289]


In [1]:
from angle_emb import AnglE, Prompts

angle = AnglE.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
# angle = AnglE.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').cuda()
angle.set_prompt(prompt=Prompts.C)
vec = angle.encode({'text': 'hello world'}, to_numpy=True)
print(vec)
vecs = angle.encode([{'text': 'hello world1'}, {'text': 'hello world2'}], to_numpy=True)
print(vecs)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


AttributeError: module 'numpy' has no attribute 'typeDict'

/opt/conda/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:511: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [3]:
angle.pooling_strategy

'cls'

In [4]:
# angle.encode('hello world')

In [5]:
#  angle.encode({'text': 'hello world'}, to_numpy=True)

In [6]:
%%time

import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define your sourceTexts and queryTexts arrays
sourceTexts = [
    "US tops 5 million confirmed virus cases",
    "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
    "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
    "The National Park Service warns against sacrificing slower friends in a bear attack",
    "Maine man wins $1M from $25 lottery ticket",
    "Make huge profits without work, earn up to $100,000 a day",
    "i want to fuck you",
    "Check your account balence",
    "you look haat"
]

queryTexts =  [
   "im so horney",
    "im feeling lucky",
    "i like nature and animals",
    "i feel cold",
    "im scared",
    "stocks"
]

# Load a pre-trained Word2Vec model (you can replace this with other embeddings models)
# Make sure you have downloaded and saved the model file locally
# model = gensim.models.Word2Vec.load('path_to_word2vec_model')

def getEmbeddingsFunction(txt):
    return angle.encode({'text': txt}, to_numpy=True)
    return  angle.encode(txt)

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Replace this with your actual getEmbeddings function
        embedding = getEmbeddingsFunction(text)
        embeddings.append(embedding)
    return embeddings

# Get embeddings for sourceTexts and queryTexts
source_embeddings = get_embeddings(sourceTexts)
query_embeddings = get_embeddings(queryTexts)

# Perform semantic search for each query
for i, query_text in enumerate(queryTexts):
    print(f"QueryText: {query_text}")
    query_vector = query_embeddings[i]

    # Calculate cosine similarity scores between the query and source texts
    similarity_scores = [cosine_similarity(query_vector, source_vector)[0][0] for source_vector in source_embeddings]

    # Sort the source texts by similarity score and get the top 3 matches
    sorted_matches = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)[:3]

    # Print the top 3 matches
    for rank, (index, score) in enumerate(sorted_matches, start=1):
        print(f"Match{rank}: {sourceTexts[index]} (Similarity Score: {score:.4f})")
    print()

QueryText: im so horney
Match1: i want to fuck you (Similarity Score: 0.4509)
Match2: you look haat (Similarity Score: 0.4087)
Match3: Check your account balence (Similarity Score: 0.4033)

QueryText: im feeling lucky
Match1: Maine man wins $1M from $25 lottery ticket (Similarity Score: 0.4928)
Match2: Make huge profits without work, earn up to $100,000 a day (Similarity Score: 0.4625)
Match3: i want to fuck you (Similarity Score: 0.4489)

QueryText: i like nature and animals
Match1: The National Park Service warns against sacrificing slower friends in a bear attack (Similarity Score: 0.5142)
Match2: i want to fuck you (Similarity Score: 0.4978)
Match3: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4017)

QueryText: i feel cold
Match1: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4358)
Match2: Check your account balence (Similarity Score: 0.4188)


In [7]:
# Perform semantic search for each query
for i, query_text in enumerate(queryTexts):
    print(f"QueryText: {query_text}")
    query_vector = query_embeddings[i]

    # Calculate cosine similarity scores between the query and source texts
    similarity_scores = cosine_similarity([query_vector], source_embeddings)

    # Sort the source texts by similarity score and get the top 3 matches
    sorted_matches = sorted(enumerate(similarity_scores[0]), key=lambda x: x[1], reverse=True)[:3]

    # Print the top 3 matches
    for rank, (index, score) in enumerate(sorted_matches, start=1):
        print(f"Match{rank}: {sourceTexts[index]} (Similarity Score: {score:.4f})")
    print()

QueryText: im so horney


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.